### LLMChains & OutputParsers

Instead of just using models you can combine a model and a prompt. This can be done with the LLMChain class.
We will additional, more complex chains in this notebook

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
TEMPLATE = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment? Sentiment is required.
subject: What subject is the text about? Use exactly one word. Use 'None' if no subject was provided.
price: How much did the customer pay? Use 'None' if no price was provided.

Format the output as JSON with the following keys:
sentiment
subject
price

text: {input}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

llm = ChatOpenAI()

prompt_template = ChatPromptTemplate.from_template(template=TEMPLATE)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.invoke(input="I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.")

### Response Schemas

There were two issues with the output: The output also contains text and the output is just a string, not a dictionary.

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="sentiment", description="is the text in a positive, neutral or negative sentiment? Sentiment is required."),
    ResponseSchema(name="subject", description="What subject is the text about? Use exactly one word. Use None if no price was provided."),
    ResponseSchema(name="price", description="How much did the customer pay? Use None if no price was provided.", type="float")
]
print(response_schemas)

In [ ]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
print(format_instructions)

In [ ]:
# Create prompt template
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Interprete the text and evaluate the text. "
            "sentiment: is the text in a positive, neutral or negative sentiment? "
            "subject: What subject is the text about? Use exactly one word. "
            "Just return the JSON, do not add ANYTHING, NO INTERPRETATION! "
            "text: {input}\n"
            "{format_instructions}\n"
        )
    ],
    input_variables=["input"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
_input = prompt.format_prompt(input="I ordered pizza salami from the restaurant Bellavista. It was ok, but the dough could have been a bit more crisp.")
output = llm.invoke(_input.to_messages())
print(output.content)

In [ ]:
json_output = parser.parse(output.content)
print(json_output)

In [ ]:
json_output.get("sentiment")